In [1]:
import json
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None

In [2]:
nan = [np.nan]
B = pd.read_excel('degree_groups.xlsx', sheet_name='B')
M = pd.read_excel('degree_groups.xlsx', sheet_name='M')
A = pd.read_excel('degree_groups.xlsx', sheet_name='A')
J = pd.read_excel('degree_groups.xlsx', sheet_name='J')
D = pd.read_excel('degree_groups.xlsx', sheet_name='D')
P = pd.read_excel('degree_groups.xlsx', sheet_name='P')
E = pd.read_excel('degree_groups.xlsx', sheet_name='E')

### Обработка json

Следующие фрагменты исправлены в json вручную:

'The University of Manchester/n1st' (li)  
'TWENTIETH CENTURY FOX\n International' (nonli)  
'WALNUT HIGH SCHOOL\nHIGH SCHOOL' (nonli)

Считываем все сырые json

In [3]:
with open('F:/Project/json/reformatted_li.json') as f:
    data = json.load(f)

with open('F:/Project/json/reformatted_nonli.json') as f:
    data1 = json.load(f)
    
with open('F:/Project/json/reformatted_nonli_extra.json') as f:
    data2 = json.load(f)
    
#print(json.dumps(data, indent=4))
#j = pd.json_normalize(data1, record_path='treasuries', meta=['uuid'])
#j

In [4]:
len(data)

7591

In [5]:
len(data1)

2028

In [6]:
len(data2)

454

Столько человек в каждом файле, суммарно - 10073

Для каждого из них есть следующий набор разделов информации. В каждом из них (кроме uuid - айдишника в базе кранчбейз и lead_general_info, которая у каждого человека единственна) содержится информация о каждом месте работе, месте учебы, etc.

In [7]:
for keys, values in data[0].items():
    print(keys, '---', type(values), '---', len(values))

lead_industries --- <class 'list'> --- 5
certifications --- <class 'list'> --- 0
educations --- <class 'list'> --- 1
honors_and_awards --- <class 'list'> --- 0
languages --- <class 'list'> --- 0
experiences --- <class 'list'> --- 6
volunteering --- <class 'list'> --- 0
publications --- <class 'list'> --- 0
treasuries --- <class 'list'> --- 0
skills --- <class 'list'> --- 0
lead_general_info --- <class 'dict'> --- 26
uuid --- <class 'str'> --- 36


Возникла следующая проблема: 2 дополнительных json, которые мы получили вдогонку, имеют лишние поля org_uuid и job_uuid, не относящиеся к LinkedIn, и поле uuid названо как person_uuid. Видимо, Андрей немного "перестарался"

In [8]:
for keys, values in data1[0].items():
    print(keys, '---', type(values), '---', len(values))

lead_industries --- <class 'list'> --- 1
certifications --- <class 'list'> --- 0
educations --- <class 'list'> --- 2
honors_and_awards --- <class 'list'> --- 0
languages --- <class 'list'> --- 0
experiences --- <class 'list'> --- 6
volunteering --- <class 'list'> --- 1
publications --- <class 'list'> --- 0
treasuries --- <class 'list'> --- 0
skills --- <class 'list'> --- 17
lead_general_info --- <class 'dict'> --- 26
person_uuid --- <class 'str'> --- 36
org_uuid --- <class 'str'> --- 36
job_uuid --- <class 'str'> --- 36


Исправим эти недоразумения, удалив лишние поля и переименовав айдишник человека. Таким образом приведем все три файла к одному виду и сможем обрабатывать их в цикле одним алгоритмом

In [9]:
for dataset in [data1, data2]:
    for person in dataset:
        del person['org_uuid']
        del person['job_uuid']
        person['uuid'] = person.pop('person_uuid')

In [10]:
for keys, values in data1[0].items():
    print(keys, '---', type(values), '---', len(values))

lead_industries --- <class 'list'> --- 1
certifications --- <class 'list'> --- 0
educations --- <class 'list'> --- 2
honors_and_awards --- <class 'list'> --- 0
languages --- <class 'list'> --- 0
experiences --- <class 'list'> --- 6
volunteering --- <class 'list'> --- 1
publications --- <class 'list'> --- 0
treasuries --- <class 'list'> --- 0
skills --- <class 'list'> --- 17
lead_general_info --- <class 'dict'> --- 26
uuid --- <class 'str'> --- 36


Далее удалим лишние места образования, которые не соответствуют получению ученой степени из выбранного перечня

In [11]:
# people = pd.read_csv('people_full.csv')
# university_degree_relation = people[['uuid', 'education_school_name', 'education_degree']]
# s1 = university_degree_relation['education_school_name'].str.split('|', expand=True).stack().str.strip().reset_index(level=1, drop=True)
# s2 = university_degree_relation['education_degree'].str.split('|', expand=True).stack().str.strip().reset_index(level=1, drop=True)
# s1 = s1.to_frame()
# s2 = s2.to_frame()
# university_degree_relation = pd.merge(s1.assign(key=s1.groupby(level=0).cumcount()).reset_index(),
#     s2.assign(key=s2.groupby(level=0).cumcount()).reset_index(),
#          how='outer',on=['index','key']).drop('key',1)
# university_degree_relation = university_degree_relation.sort_values(by = ['index']).rename(columns={'index': 'person', '0_x': 'name', '0_y': 'degree'}).reset_index(drop=True)
# university_degree_relation = university_degree_relation.groupby(['name', 'degree']).size().reset_index(name = 'count')
# university_degree_relation = university_degree_relation.sort_values(by = ['count'], ascending=False)
# university_degree_relation.loc[university_degree_relation['degree'].isin(B['degrees'].values), 'degree_class'] = 'B'
# university_degree_relation.loc[university_degree_relation['degree'].isin(M['degrees'].values), 'degree_class'] = 'M'
# university_degree_relation.loc[university_degree_relation['degree'].isin(A['degrees'].values), 'degree_class'] = 'A'
# university_degree_relation.loc[university_degree_relation['degree'].isin(J['degrees'].values), 'degree_class'] = 'J'
# university_degree_relation.loc[university_degree_relation['degree'].isin(D['degrees'].values), 'degree_class'] = 'D'
# university_degree_relation.loc[university_degree_relation['degree'].isin(P['degrees'].values), 'degree_class'] = 'P'
# university_degree_relation.loc[university_degree_relation['degree'].isin(E['degrees'].values), 'degree_class'] = 'E'
# university_degree_relation.to_csv('university_degree_relation.csv', index=False, encoding = 'utf-8-sig')
# university_degree_relation

Здесь было 11095 строк, их обработали вручную, расставив пометки школ, финансистов и акселераторов

Дополнили остальные степени вручную и удалили Schools, Other и Accelerators

In [12]:
ufg = pd.read_csv('university_family_groups.csv', sep=';')
ufg = ufg[['name', 'degree', 'degree_class', 'count', 'family']]
ufg = ufg.loc[~((ufg['degree_class'] == 'S') | (ufg['degree_class'] == 'O') | (ufg['degree_class'] == 'Y'))]
ufg

,name,degree,degree_class,count,family
0,Harvard Business School,MBA,A,437,Harvard
1,Stanford University Graduate School of Business,MBA,A,294,Stanford
2,The Wharton School,MBA,A,202,UPenn
3,Stanford University,MS,M,107,Stanford
4,Stanford University,BS,B,105,Stanford
...,...,...,...,...,...
11087,Zhejiang University,Dual Degree,B,1,NaN
11088,Zicklin School of Business,Bachelor of Business Administration (BBA),B,1,NaN
11091,清华五道口金融学院,硕士,M,1,Tsinghua
11092,清华大学五道口金融学院,EMBA,E,1,Tsinghua


In [13]:
ufg2 = ufg.copy()
ufg2 = ufg2.loc[~ufg2['family'].isin(nan)]
#ufg2.loc[ufg2['family'].isin(nan), 'family'] = ufg2['name']
#ufg2.loc[ufg2['name'].isin(nan), 'family'] = 0
ufg2 = ufg2.groupby(['family', 'degree_class'])['count'].agg(['sum']).reset_index() 
ufg2 = ufg2.sort_values(by = ['sum'], ascending=False).reset_index(drop=True)
tab = pd.crosstab(index = ufg2['family'], columns = ufg2['degree_class'], values = ufg2['sum'], aggfunc='sum', dropna = False)
tab.to_csv('degree_distrib.csv', index=False, encoding = 'utf-8-sig')
tab

degree_class,A,B,C,D,E,J,M,P
family,,,,,,,,
ASU,4.0,30.0,NaN,NaN,1.0,NaN,3.0,1.0
Amherst,NaN,38.0,NaN,NaN,NaN,NaN,NaN,NaN
BU,24.0,42.0,NaN,1.0,1.0,8.0,8.0,2.0
BYU,4.0,40.0,NaN,NaN,NaN,4.0,4.0,NaN
Babson,9.0,29.0,NaN,NaN,1.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...
WUSTL,5.0,27.0,NaN,8.0,1.0,1.0,11.0,5.0
Waterloo,NaN,28.0,NaN,NaN,NaN,NaN,4.0,1.0
Wesleyan,NaN,37.0,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
ufg2 = ufg.copy()
ufg2 = ufg2.loc[~ufg2['family'].isin(nan)]
#ufg2.loc[ufg2['family'].isin(nan), 'family'] = ufg2['name']
#ufg2.loc[ufg2['name'].isin(nan), 'family'] = 0
ufg2 = ufg2.groupby(['family'])['count'].agg(['sum']).reset_index() 
ufg2 = ufg2.sort_values(by = ['sum'], ascending=False).reset_index(drop=True)
ufg2

,family,sum
0,Stanford,1511
1,Harvard,1425
2,UPenn,695
3,MIT,541
4,Berkeley,504
...,...,...
108,TCDublin,20
109,CMC,20
110,Bowdoin,19
111,PomonaCollege,18


Изменили логику. Вместо разбиения на 5 классов сделаем 2 класса - топ-5 вузов (флаг 1) и все остальные (флаг 0)

In [15]:
ufg2['top_quantil'] = '---'

ufg2.at[0:4, 'top_quantil'] = 1
#ufg2.at[0:1, 'top_quantil'] = 'group_a'
#ufg2.at[2:9, 'top_quantil'] = 'group_b'
#ufg2.at[10:25, 'top_quantil'] = 'group_c'
#ufg2.at[26:112, 'top_quantil'] = 'group_d'

ufg2 = ufg2[['family', 'top_quantil']]
ufg2.loc[[4]]

,family,top_quantil
4,Berkeley,1


Добавим флаги is_PHD, is_MBA и is_Master для докторских и магистерских степеней

In [16]:
ufg = ufg.merge(ufg2, how = 'left', on = 'family')

ufg.loc[ufg['top_quantil'] != 1, 'top_quantil'] = 0
ufg.loc[ufg['degree_class'] == 'P', 'is_PHD'] = 1
ufg.loc[ufg['degree_class'] == 'M', 'is_Master'] = 1
ufg.loc[ufg['degree_class'] == 'A', 'is_MBA'] = 1
ufg.loc[ufg['degree_class'] != 'P', 'is_PHD'] = 0
ufg.loc[ufg['degree_class'] != 'M', 'is_Master'] = 0
ufg.loc[ufg['degree_class'] != 'A', 'is_MBA'] = 0
#ufg.loc[ufg['top_quantil'].isin(nan), 'top_quantil'] = 'group_outsider'
#ufg["top_quantil"] = ufg["top_quantil"] + '_' + ufg["degree_class"]

ufg.to_csv('ufg.csv', index=False, encoding = 'utf-8-sig')
ufg

,name,degree,degree_class,count,family,top_quantil,is_PHD,is_Master,is_MBA
0,Harvard Business School,MBA,A,437,Harvard,1,0.0,0.0,1.0
1,Stanford University Graduate School of Business,MBA,A,294,Stanford,1,0.0,0.0,1.0
2,The Wharton School,MBA,A,202,UPenn,1,0.0,0.0,1.0
3,Stanford University,MS,M,107,Stanford,1,0.0,1.0,0.0
4,Stanford University,BS,B,105,Stanford,1,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
8609,Zhejiang University,Dual Degree,B,1,NaN,0,0.0,0.0,0.0
8610,Zicklin School of Business,Bachelor of Business Administration (BBA),B,1,NaN,0,0.0,0.0,0.0
8611,清华五道口金融学院,硕士,M,1,Tsinghua,0,0.0,1.0,0.0
8612,清华大学五道口金融学院,EMBA,E,1,Tsinghua,0,0.0,0.0,0.0


In [20]:
ufg_new = ufg.groupby(['family', 'degree_class', 'degree'])['count'].agg(['sum']).reset_index()
ufg_new.to_csv('ufg_new.csv', encoding = 'utf-8-sig')
ufg_new

,family,degree_class,degree,sum
0,ASU,A,MBA,2
1,ASU,A,Master of Business Administration (MBA),1
2,ASU,A,Master of Business Administration - MBA,1
3,ASU,B,B.S.,3
4,ASU,B,BA,2
...,...,...,...,...
3824,Yale,P,Doctor of Philosophy (PhD),1
3825,Yale,P,"M.S., M.Phil., Ph.D.",1
3826,Yale,P,Ph.D.,4
3827,Yale,P,PhD,1


### Основная таблица

Следующий блок совмещает всю информацию из 3 датасетов в виде датафрейма, где каждая строчка - уникальный человек, а в последующих столбцах через разделитель | перечислены сферы индустрии, места образования и места работы

In [17]:
df = pd.DataFrame()

for dataset in [data, data1, data2]:
    a = pd.json_normalize(dataset)
    a = a.loc[:, 'uuid':]

    #b = pd.json_normalize(person, record_path='lead_industries', meta=['uuid'])
    #c = pd.json_normalize(person, record_path='certifications', meta=['uuid'])
    
    d = pd.json_normalize(dataset, record_path='educations', meta=['uuid'])
    d = d.merge(ufg[['name', 'degree', 'top_quantil', 'is_PHD', 'is_Master', 'is_MBA']], how = 'left', left_on=['education_school_name', 'education_degree'], right_on=['name', 'degree'])
    
    #расставляем флаги, без склеивания
    #d = d.groupby('uuid').agg(lambda x: '|'.join(list(x.dropna()))).reset_index()
    d['top_quantil'] = d['top_quantil'].replace(np.nan, 0)
    d['is_PHD'] = d['is_PHD'].replace(np.nan, 0)
    d['is_Master'] = d['is_Master'].replace(np.nan, 0)
    d['is_MBA'] = d['is_MBA'].replace(np.nan, 0)
    d = d.groupby(['uuid'])['top_quantil', 'is_PHD', 'is_Master', 'is_MBA'].agg('max').reset_index()
    
    #e = pd.json_normalize(person, record_path='honors_and_awards', meta=['uuid'])
    #f = pd.json_normalize(person, record_path='languages', meta=['uuid'])
    
    g = pd.json_normalize(dataset, record_path='experiences', meta=['uuid'])
    g = g.groupby('uuid').agg(lambda x: '|'.join(list(x.dropna()))).reset_index()
    
    #h = pd.json_normalize(person, record_path='volunteering', meta=['uuid'])
    #i = pd.json_normalize(person, record_path='publications', meta=['uuid'])
    #j = pd.json_normalize(person, record_path='treasuries', meta=['uuid'])
    #k = pd.json_normalize(person, record_path='skills', meta=['uuid'])
    
    #a = a.merge(b, how = 'left', on = 'uuid')
    #a = a.merge(c, how = 'left', on = 'uuid')
    a = a.merge(d, how = 'left', on = 'uuid')
    #a = a.merge(e, how = 'left', on = 'uuid')
    #a = a.merge(f, how = 'left', on = 'uuid')
    a = a.merge(g, how = 'left', on = 'uuid')
    #a = a.merge(h, how = 'left', on = 'uuid')
    #a = a.merge(i, how = 'left', on = 'uuid')
    #a = a.merge(j, how = 'left', on = 'uuid')
    #a = a.merge(k, how = 'left', on = 'uuid')
    
    df = pd.concat([df, a], ignore_index=True)
    df['top_quantil'] = df['top_quantil'].replace(np.nan, 0)
    df['is_PHD'] = df['is_PHD'].replace(np.nan, 0)
    df['is_Master'] = df['is_Master'].replace(np.nan, 0)
    df['is_MBA'] = df['is_MBA'].replace(np.nan, 0)
    
df[['uuid', 'lead_general_info.lead_full_name', 'top_quantil', 'is_PHD', 'is_Master', 'is_MBA', 'experience_company_name']]

<ipython-input-17-2174d081dd02>:19: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  d = d.groupby(['uuid'])['top_quantil', 'is_PHD', 'is_Master', 'is_MBA'].agg('max').reset_index()


,uuid,lead_general_info.lead_full_name,top_quantil,is_PHD,is_Master,is_MBA,experience_company_name
0,001e860c-6579-9835-b294-af1a65842ca9,Evan Rifkin,0.0,0.0,0.0,0.0,"Aircam|Apple|Burstly|Flux, a Social Project|TagWorld|Adconion Media Group"
1,0023bf7f-5417-04f4-85cf-5bb078576a57,Jaynie Miller Studenmund,1.0,0.0,0.0,1.0,Pacific Premier Bancorp/Bank (PPBI)|EXL Service (EXLS)|LifeLock (LOCK)|Enduring Heroes Foundation (Non-profit)|Pinnacle Entertainment (PNK)|CoreLogic (CLGX)|Orbitz Worldwide (OWW)|Western Asset Management (WAMCO)|Forest Lawn Memorial Parks (Non-profit)|Huntington Hospital (Non-profit)
2,0026bf40-8f42-87e2-b371-978055165959,Josh Elman,1.0,0.0,0.0,1.0,"Apple|Robinhood|Greylock Partners|Mammoth Media|Discord|musical.ly Inc.|Houseparty (part of Epic Games)|Medium|Greylock Partners|Nextdoor, Inc.|Greylock Partners|."
3,0029836e-5dec-d285-c440-00d33464ede2,Vipul Ved Prakash,0.0,0.0,0.0,0.0,Apple|Topsy Labs|Napster|Cloudmark|Tradeindia.com - Infocom Network Ltd|Sense/Net
4,003bbe2e-18be-3c96-e327-18d2295427e0,Larry Cheng,1.0,0.0,0.0,0.0,Volition Capital|Fidelity Ventures|Battery Ventures|Bessemer Venture Partners|Corporate Decisions Inc. / Mercer Management Consulting|Harvard Student Agencies
...,...,...,...,...,...,...,...
10068,df78fb12-8ec7-9208-b8a7-5abed3117746,Kim Perell,0.0,0.0,0.0,0.0,100.co|Entrepreneur Media|SmartMedia Technologies|Perell Ventures|Turn|Amobee|Amobee|Adconion Direct|Frontline Direct|Xdrive Technologies
10069,ab6e59e9-163d-5a50-a0a3-b30e7d822d72,Hans Bishop,0.0,0.0,1.0,0.0,Benchmarc Display Inc.|Benchmarc Display Inc.|Benchmarc Display Inc.|Benchmarc Display Inc.|Thomas A. Schutz Corporation
10070,767846ae-f695-72b5-ba65-f9e50cebf800,David Duffield,0.0,0.0,0.0,0.0,Workday
10071,5ccfa9c5-3c80-596f-9e4a-f76b1a12047e,Bob Whitehead,0.0,0.0,0.0,0.0,Mona Vie|Delaware Express Shuttle|School District of Philadelphia


Расстановка статусов предприниматель/инвестор/и тот и тот, а также лучших ученых степеней

In [18]:
list_of_entrepreneurs = pd.read_csv('./list_of_entrepreneurs.csv')
entrepreneurs = set(list_of_entrepreneurs['person_uuid'].values)
len(entrepreneurs)

5250

In [19]:
list_of_investors = pd.read_csv('./list_of_investors.csv')
investors = set(list_of_investors['person_uuid'].values)
len(investors)

6390

In [20]:
caesars = entrepreneurs.intersection(investors)
len(caesars)

1024

In [21]:
df.loc[df['uuid'].isin(entrepreneurs), 'status'] = 'entrepreneur'
df.loc[df['uuid'].isin(investors), 'status'] = 'investor'
df.loc[df['uuid'].isin(caesars), 'status'] = 'caesar'
entrepreneurs = df.loc[df['status'] == 'entrepreneur']
investors = df.loc[df['status'] == 'investor']
caesars = df.loc[df['status'] == 'caesar']

In [22]:
df2 = pd.DataFrame()
for dataset in [data, data1, data2]:
    a = pd.json_normalize(dataset, record_path='educations', meta=['uuid'])
    df2 = pd.concat([df2, a], ignore_index=True)

df2.loc[df2['education_degree'].isin(E['degrees'].values), 'executive_degree'] = 'E'
a = df2.loc[df2['executive_degree'] == 'E']
df2.loc[df2['uuid'].isin(a['uuid'].values), 'executive_degree'] = 'E'
df2.loc[df2['education_degree'].isin(B['degrees'].values), 'best_degree'] = 1
df2.loc[df2['education_degree'].isin(M['degrees'].values), 'best_degree'] = 2
df2.loc[df2['education_degree'].isin(A['degrees'].values), 'best_degree'] = 3
df2.loc[df2['education_degree'].isin(J['degrees'].values), 'best_degree'] = 4
df2.loc[df2['education_degree'].isin(D['degrees'].values), 'best_degree'] = 5
df2.loc[df2['education_degree'].isin(P['degrees'].values), 'best_degree'] = 6
df2 = df2.loc[~df2['best_degree'].isin(nan)]
df2 = df2.loc[df2.groupby(['uuid'])['best_degree'].idxmax()]
df2 = df2[['uuid', 'executive_degree', 'best_degree']]
df2.loc[df2['best_degree'] == 1, 'best_degree'] = 'B'
df2.loc[df2['best_degree'] == 2, 'best_degree'] = 'M'
df2.loc[df2['best_degree'] == 3, 'best_degree'] = 'A'
df2.loc[df2['best_degree'] == 4, 'best_degree'] = 'J'
df2.loc[df2['best_degree'] == 5, 'best_degree'] = 'D'
df2.loc[df2['best_degree'] == 6, 'best_degree'] = 'P'

df = df.merge(df2, how = 'left', on = 'uuid')
df.to_csv('people_full.csv', index=False, encoding = 'utf-8-sig')
df

,uuid,lead_general_info.lead_headline_location_country,lead_general_info.lead_first_name,lead_general_info.lead_last_name,lead_general_info.lead_full_name,lead_general_info.lead_birth_date,lead_general_info.lead_linkedin_member_id,lead_general_info.lead_state_what,lead_general_info.lead_public_identifier,lead_general_info.lead_tracking_id,lead_general_info.lead_associated_hashtags_copy,lead_general_info.lead_memorialized_bool,lead_general_info.lead_multi_locale_phonetic_first_name,lead_general_info.lead_volunteer_causes,lead_general_info.lead_temp_status_emoji,lead_general_info.lead_maiden_name,lead_general_info.lead_geo_location_backfilled_bool,lead_general_info.lead_phonetic_last_name,lead_general_info.lead_premium_bool,lead_general_info.lead_influencer_bool,lead_general_info.lead_headline,lead_general_info.lead_full_name_pronunciation_audio,lead_general_info.lead_summary,lead_general_info.lead_location_name,lead_general_info.lead_address,lead_general_info.lead_headline_location_without_country,lead_general_info.lead_headline_location_with_country,lead_general_info.lead_full_name_pronunciation_audio.urn,lead_general_info.lead_full_name_pronunciation_audio.duration,lead_general_info.lead_full_name_pronunciation_audio.url,lead_general_info.lead_full_name_pronunciation_audio.$type,lead_general_info.lead_birth_date.month,lead_general_info.lead_birth_date.day,lead_general_info.lead_birth_date.$type,lead_general_info.lead_birth_date.year,lead_general_info.lead_multi_locale_phonetic_first_name.en_US,lead_general_info.lead_multi_locale_phonetic_first_name.ja_JP,lead_general_info.lead_state_what.stateCode,lead_general_info.lead_state_what.stateName,lead_general_info.lead_state_what.$type,top_quantil,is_PHD,is_Master,is_MBA,experience_title,experience_start_date,experience_start_date_year,experience_end_date,experience_headcount_range,experience_company_name,experience_company_link,experience_company_id_link,experience_industry,experience_location,experience_description,status,executive_degree,best_degree
0,001e860c-6579-9835-b294-af1a65842ca9,,Evan,Rifkin,Evan Rifkin,,41865,,evanrifkin,NPRfcMTmSZ+PC+Of4bNO/w==,,False,,,,,False,,False,False,"Co-Founder, CEO at Aircam",,,,,"Santa Monica, California","Santa Monica, California, United States",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,"Co-Founder, CEO|Project Team Lead|Co-Founder|Co-Founder|Co-Founder|Founding Investor",Jan 2018|Feb 2014|Apr 2009|Dec 2006|May 2005|2005,2018|2014|2009|2006|2005|2005,|Oct 2014|Feb 2014|Dec 2008|Dec 2006|2006,11-50|10001|11-50||51-200|501-1000,"Aircam|Apple|Burstly|Flux, a Social Project|TagWorld|Adconion Media Group",https://www.linkedin.com/company/aircam-inc/|https://www.linkedin.com/company/apple/|https://www.linkedin.com/company/burstly/||https://www.linkedin.com/company/tagworld/|https://www.linkedin.com/company/adconion-media-group/,https://www.linkedin.com/company/19015454|https://www.linkedin.com/company/162479|https://www.linkedin.com/company/1824649|https://www.linkedin.com/company/|https://www.linkedin.com/company/60839|https://www.linkedin.com/company/30596,4|24|80||6|113,|||||,"|||Social Project was sold to MTVN/Viacom in September 2008.\nSocial Project created the Flux Platform, enabling publishers to add social media tools and distribute content across different partner sites of Flux.|TagWorld became Social Project/Flux after an investment from MTVN/Viacom in December 2006.|",entrepreneur,NaN,B
1,0023bf7f-5417-04f4-85cf-5bb078576a57,,Jaynie Miller,Studenmund,Jaynie Miller Studenmund,,83202432,,jaynie-miller-studenmund-680a4523,NN4yJ60NSGe7yg30QoP6gQ==,,False,,,,,False,,False,False,"Corporate Director, Retired COO, Gold Star Mother",,"Jaynie is a seasoned, goal oriented business leader who is engaged with a portfolio of corporate and non profit boards. She has served on 7 public, 3 private, and numerous non profit boards. \n\nJaynie’s board service leverages 30 years of line operating, P&L experience in C-suite positi

### Распределения мест работы, учебы и степеней

In [23]:
# university_urn_relation = df[['uuid', 'education_school_name', 'education_school_urn']]
# s1 = university_urn_relation['education_school_name'].str.split('|', expand=True).stack().str.strip().reset_index(level=1, drop=True)
# s2 = university_urn_relation['education_school_urn'].str.split('|', expand=True).stack().str.strip().reset_index(level=1, drop=True)
# s1 = s1.to_frame()
# s2 = s2.to_frame()
# university_urn_relation = pd.merge(s1.assign(key=s1.groupby(level=0).cumcount()).reset_index(),
#     s2.assign(key=s2.groupby(level=0).cumcount()).reset_index(),
#          how='outer',on=['index','key']).drop('key',1)
# university_urn_relation = university_urn_relation.sort_values(by = ['index']).rename(columns={'index': 'person', '0_x': 'name', '0_y': 'urn'}).reset_index(drop=True)
# university_urn_relation = university_urn_relation.groupby(['name', 'urn']).size().reset_index(name = 'count')
# university_urn_relation = university_urn_relation.sort_values(by = ['count'], ascending=False)
# university_urn_relation.to_csv('university_urn_relation.csv', index=False, encoding = 'utf-8-sig')
# university_urn_relation

### Entrepreneurs

In [24]:
# entrepreneurs_exp = pd.DataFrame(entrepreneurs['experience_company_name'].str.split('|', expand=True).values.tolist(), index=entrepreneurs['uuid']).stack()
# entrepreneurs_exp = entrepreneurs_exp.reset_index()[[0, 'uuid']]
# entrepreneurs_exp.columns = ['experience_company_name', 'uuid']
# entrepreneurs_exp = entrepreneurs_exp.groupby(['experience_company_name']).size().reset_index(name = 'count')
# entrepreneurs_exp = entrepreneurs_exp.sort_values(by = ['count'], ascending=False)
# entrepreneurs_exp.to_csv('entrepreneurs_experience.csv', index=False, encoding = 'utf-8-sig')
# entrepreneurs_exp.head(5)

In [25]:
# entrepreneurs_deg = pd.DataFrame(entrepreneurs['education_degree'].str.split('|', expand=True).values.tolist(), index=entrepreneurs['uuid']).stack()
# entrepreneurs_deg = entrepreneurs_deg.reset_index()[[0, 'uuid']]
# entrepreneurs_deg.columns = ['education_degree', 'uuid']
# entrepreneurs_deg = entrepreneurs_deg.groupby(['education_degree']).size().reset_index(name = 'count')
# entrepreneurs_deg = entrepreneurs_deg.sort_values(by = ['count'], ascending=False)
# entrepreneurs_deg.loc[entrepreneurs_deg['education_degree'].isin(E['degrees'].values), 'best_degree'] = 'E'
# entrepreneurs_deg.loc[entrepreneurs_deg['education_degree'].isin(B['degrees'].values), 'best_degree'] = 'B'
# entrepreneurs_deg.loc[entrepreneurs_deg['education_degree'].isin(M['degrees'].values), 'best_degree'] = 'M'
# entrepreneurs_deg.loc[entrepreneurs_deg['education_degree'].isin(A['degrees'].values), 'best_degree'] = 'A'
# entrepreneurs_deg.loc[entrepreneurs_deg['education_degree'].isin(J['degrees'].values), 'best_degree'] = 'J'
# entrepreneurs_deg.loc[entrepreneurs_deg['education_degree'].isin(D['degrees'].values), 'best_degree'] = 'D'
# entrepreneurs_deg.loc[entrepreneurs_deg['education_degree'].isin(P['degrees'].values), 'best_degree'] = 'P'
# entrepreneurs_deg.to_csv('entrepreneurs_degrees.csv', index=False, encoding = 'utf-8-sig')
# entrepreneurs_deg.head(5)

In [26]:
# entrepreneurs_ed = pd.DataFrame(entrepreneurs['education_school_name'].str.split('|', expand=True).values.tolist(), index=entrepreneurs['uuid']).stack()
# entrepreneurs_ed = entrepreneurs_ed.reset_index()[[0, 'uuid']]
# entrepreneurs_ed.columns = ['education_school_name', 'uuid']
# entrepreneurs_ed = entrepreneurs_ed.groupby(['education_school_name']).size().reset_index(name = 'count')
# entrepreneurs_ed = entrepreneurs_ed.sort_values(by = ['count'], ascending=False)
# entrepreneurs_ed.to_csv('entrepreneurs_education.csv', index=False, encoding = 'utf-8-sig')
# entrepreneurs_ed.head(5)

### Investors

In [27]:
# investors_exp = pd.DataFrame(investors['experience_company_name'].str.split('|', expand=True).values.tolist(), index=investors['uuid']).stack()
# investors_exp = investors_exp.reset_index()[[0, 'uuid']]
# investors_exp.columns = ['experience_company_name', 'uuid']
# investors_exp = investors_exp.groupby(['experience_company_name']).size().reset_index(name = 'count')
# investors_exp = investors_exp.sort_values(by = ['count'], ascending=False)
# investors_exp.to_csv('investors_experience.csv', index=False, encoding = 'utf-8-sig')
# investors_exp.head(5)

In [28]:
# investors_deg = pd.DataFrame(investors['education_degree'].str.split('|', expand=True).values.tolist(), index=investors['uuid']).stack()
# investors_deg = investors_deg.reset_index()[[0, 'uuid']]
# investors_deg.columns = ['education_degree', 'uuid']
# investors_deg = investors_deg.groupby(['education_degree']).size().reset_index(name = 'count')
# investors_deg = investors_deg.sort_values(by = ['count'], ascending=False)
# investors_deg.loc[investors_deg['education_degree'].isin(E['degrees'].values), 'best_degree'] = 'E'
# investors_deg.loc[investors_deg['education_degree'].isin(B['degrees'].values), 'best_degree'] = 'B'
# investors_deg.loc[investors_deg['education_degree'].isin(M['degrees'].values), 'best_degree'] = 'M'
# investors_deg.loc[investors_deg['education_degree'].isin(A['degrees'].values), 'best_degree'] = 'A'
# investors_deg.loc[investors_deg['education_degree'].isin(J['degrees'].values), 'best_degree'] = 'J'
# investors_deg.loc[investors_deg['education_degree'].isin(D['degrees'].values), 'best_degree'] = 'D'
# investors_deg.loc[investors_deg['education_degree'].isin(P['degrees'].values), 'best_degree'] = 'P'
# investors_deg.to_csv('investors_degrees.csv', index=False, encoding = 'utf-8-sig')
# investors_deg.head(5)

In [29]:
# investors_ed = pd.DataFrame(investors['education_school_name'].str.split('|', expand=True).values.tolist(), index=investors['uuid']).stack()
# investors_ed = investors_ed.reset_index()[[0, 'uuid']]
# investors_ed.columns = ['education_school_name', 'uuid']
# investors_ed = investors_ed.groupby(['education_school_name']).size().reset_index(name = 'count')
# investors_ed = investors_ed.sort_values(by = ['count'], ascending=False)
# investors_ed.to_csv('investors_education.csv', index=False, encoding = 'utf-8-sig')
# investors_ed.head(5)

### Caesars

In [30]:
# caesars_exp = pd.DataFrame(caesars['experience_company_name'].str.split('|', expand=True).values.tolist(), index=caesars['uuid']).stack()
# caesars_exp = caesars_exp.reset_index()[[0, 'uuid']]
# caesars_exp.columns = ['experience_company_name', 'uuid']
# caesars_exp = caesars_exp.groupby(['experience_company_name']).size().reset_index(name = 'count')
# caesars_exp = caesars_exp.sort_values(by = ['count'], ascending=False)
# caesars_exp.to_csv('caesars_experience.csv', index=False, encoding = 'utf-8-sig')
# caesars_exp.head(5)

In [31]:
# caesars_deg = pd.DataFrame(caesars['education_degree'].str.split('|', expand=True).values.tolist(), index=caesars['uuid']).stack()
# caesars_deg = caesars_deg.reset_index()[[0, 'uuid']]
# caesars_deg.columns = ['education_degree', 'uuid']
# caesars_deg = caesars_deg.groupby(['education_degree']).size().reset_index(name = 'count')
# caesars_deg = caesars_deg.sort_values(by = ['count'], ascending=False)
# caesars_deg.loc[caesars_deg['education_degree'].isin(E['degrees'].values), 'best_degree'] = 'E'
# caesars_deg.loc[caesars_deg['education_degree'].isin(B['degrees'].values), 'best_degree'] = 'B'
# caesars_deg.loc[caesars_deg['education_degree'].isin(M['degrees'].values), 'best_degree'] = 'M'
# caesars_deg.loc[caesars_deg['education_degree'].isin(A['degrees'].values), 'best_degree'] = 'A'
# caesars_deg.loc[caesars_deg['education_degree'].isin(J['degrees'].values), 'best_degree'] = 'J'
# caesars_deg.loc[caesars_deg['education_degree'].isin(D['degrees'].values), 'best_degree'] = 'D'
# caesars_deg.loc[caesars_deg['education_degree'].isin(P['degrees'].values), 'best_degree'] = 'P'
# caesars_deg.to_csv('caesars_degrees.csv', encoding = 'utf-8-sig')
# caesars_deg.head(5)

In [32]:
# caesars_ed = pd.DataFrame(caesars['education_school_name'].str.split('|', expand=True).values.tolist(), index=caesars['uuid']).stack()
# caesars_ed = caesars_ed.reset_index()[[0, 'uuid']]
# caesars_ed.columns = ['education_school_name', 'uuid']
# caesars_ed = caesars_ed.groupby(['education_school_name']).size().reset_index(name = 'count')
# caesars_ed = caesars_ed.sort_values(by = ['count'], ascending=False)
# caesars_ed.to_csv('caesars_education.csv', index=False, encoding = 'utf-8-sig')
# caesars_ed.head(5)

### Соединим людей с их работой

In [33]:
jobs = pd.read_csv('F:/Project/csv/jobs.csv')
investments = pd.read_csv('F:/Project/csv/investments.csv')

funding_rounds = pd.read_csv('C:/Users/Barca/Desktop/GitHub/IPO_Analysis/model_ver_based_on_2019/wave_1/1_DataPreparation/funding_rounds_with_exits.csv')
inv_list = pd.read_csv('C:/Users/Barca/Desktop/GitHub/IPO_Analysis/model_ver_based_on_2019/wave_1/1_DataPreparation/investor_list.csv')

Для наших 10073 человек найдем все позиции/вакансии, связанные либо со стартапом из выбранных 736, либо с фондом из выбранных 211.  
Соединим эти позиции с названиями стартапов и фондов

In [34]:
cb_jobs = jobs.loc[jobs['person_uuid'].isin(df['uuid']) & (jobs['org_uuid'].isin(funding_rounds['company_uuid'].values) | (jobs['org_uuid'].isin(inv_list['investor_uuid'])))]
s = funding_rounds.groupby(['company_name', 'company_uuid']).size().reset_index(name = 'count')
s = s[['company_name', 'company_uuid']]
f = inv_list[['investor_name', 'investor_uuid']]
cb_jobs = cb_jobs.merge(s, how = 'left', left_on = 'org_uuid', right_on = 'company_uuid')
cb_jobs = cb_jobs.merge(f, how = 'left', left_on = 'org_uuid', right_on = 'investor_uuid')
cb_jobs

,person_uuid,org_uuid,job_uuid,started_on,ended_on,is_current,title,job_type,company_name,company_uuid,investor_name,investor_uuid
0,5fa59ac4-e52c-cff6-520d-b56b1b19016e,2d0a5bf0-e53e-9a07-2243-9bcd561421ec,e5d422fe-d875-e5e1-59bf-d07576ffae7d,2005-01-01,NaN,t,General Partner,executive,NaN,NaN,Kleiner Perkins Caufield & Byers,2d0a5bf0-e53e-9a07-2243-9bcd561421ec
1,62d89c9a-87f9-b7f9-5379-dfe45b99780d,e0906c05-fae5-9591-ba5f-2142d8b0065a,b931ab6a-e8aa-5a21-6b27-b3b22f23323f,2002-01-01,2009-01-01,f,Product Management,executive,NaN,NaN,Cisco,e0906c05-fae5-9591-ba5f-2142d8b0065a
2,62d89c9a-87f9-b7f9-5379-dfe45b99780d,71c2f7e6-759d-6f24-d0b9-7241f1488a12,97bebf43-96fc-449f-93c2-634fd998341a,2013-01-01,NaN,t,Board Member,board_member,Versa Networks,71c2f7e6-759d-6f24-d0b9-7241f1488a12,NaN,NaN
3,77c94eb8-b122-8b7b-0ce4-bae96a68f1af,7f5fee94-6336-dcde-e6df-2fc7ed1951ab,6981631d-ad11-443e-a96b-cddf616bef93,NaN,NaN,t,Director of Engineering,executive,CloudBees,7f5fee94-6336-dcde-e6df-2fc7ed1951ab,NaN,NaN
4,52c4e0cd-abbd-4e32-acd8-e196899b58ff,8c725124-83b2-cd3f-9af3-4117ac81f75b,991edbfa-ccf6-4146-825a-61d2bf109551,2013-12-01,NaN,t,CTO,executive,Business Insider,8c725124-83b2-cd3f-9af3-4117ac81f75b,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13932,c4ce1ac3-85b8-408e-b06c-05bdcb7b956d,30b3efdf-6024-804d-ea55-c4d1a8dfc86c,70f798e1-7684-4c31-be1c-a436c6b355ad,2011-05-01,2015-05-01,f,Vice President,employee,NaN,NaN,Goldman Sachs,30b3efdf-6024-804d-ea55-c4d1a8dfc86c
13933,f3029096-9edd-8535-f0df-669ff0c60839,c30618c2-6759-3ce8-0535-35d3b4216fd6,a3d5d779-a3d3-4e8a-b8d9-f4aa7d25476e,2017-12-01,NaN,t,General Manager,executive,Upland Qvidian,c30618c2-6759-3ce8-0535-35d3b4216fd6,NaN,NaN
13934,0cffa432-ea89-49b3-8c20-d3dc6fdb4b41,2613693e-8757-5d7d-a565-0a5052ecde76,89aaf42e-cc38-4423-b577-0720e28fee88,NaN,NaN,t,Partner,executive,NaN,NaN,Generation Investment Management,2613693e-8757-5d7d-a565-0a5052ecde76
13935,d2ca0499-eda5-4b1d-9abc-5bd664881c87,04850d4f-10e8-1f2f-e54a-2b9ceae952d5,eb699432-52af-4f0a-9319-004f0a5d119e,NaN,NaN,t,Investment Director,executive,NaN,NaN,Qualcomm Ventures,04850d4f-10e8-1f2f-e54a-2b9ceae952d5


Столько интересующих нас позиций в фондах и стартапах есть у выбранных людей.  
С помощью группировки определили, что здесь представлены 720 стартапов и 200 фондов (из 736 и 211 соответственно)

Некоторые из людей имеют по нескольку вакансий в этом списке (например разные позиции в одном стартапе)

In [35]:
cb_jobs.groupby(['person_uuid', 'company_uuid']).size().reset_index(name = 'count')

,person_uuid,company_uuid,count
0,0014409b-f138-ecd1-31ad-6096989254c3,b6e1f019-42f5-7259-dc87-0de0ea01ae17,1
1,001e860c-6579-9835-b294-af1a65842ca9,997b1e1e-226e-16e1-c977-a0ac3ed940b1,1
2,0023bf7f-5417-04f4-85cf-5bb078576a57,b93d25ff-cf38-929e-57a6-b40bf77db9ca,1
3,0026bf40-8f42-87e2-b371-978055165959,3e2a5b75-1241-ce29-930a-7d616d5c3a5e,1
4,0029836e-5dec-d285-c440-00d33464ede2,c824b0c6-f584-87af-5475-f4962effe1df,1
...,...,...,...
5967,ff94aaa0-dc72-6baa-13a5-4fbe22b3b237,f36c0dc1-80ae-c232-0144-b62952ba5096,1
5968,ffbde8a6-d828-b2fb-c285-729ad1733c15,a6946400-ce50-194b-fdeb-06edc1d538ad,1
5969,ffd2285d-cda7-30de-2ae2-c37b7956ecd4,77f5e56b-859f-f1b1-88a8-fbcd945101f1,1
5970,ffdfc425-c8da-8a83-32d5-e64d18166bd1,f2c269cc-5eae-129f-30c9-40d9cba1eab4,1


Соединим вакансии из кранчбейза и нашу основную таблицу с данными из LI

In [36]:
df = df.merge(cb_jobs, how = 'left', left_on = 'uuid', right_on = 'person_uuid')
df

,uuid,lead_general_info.lead_headline_location_country,lead_general_info.lead_first_name,lead_general_info.lead_last_name,lead_general_info.lead_full_name,lead_general_info.lead_birth_date,lead_general_info.lead_linkedin_member_id,lead_general_info.lead_state_what,lead_general_info.lead_public_identifier,lead_general_info.lead_tracking_id,lead_general_info.lead_associated_hashtags_copy,lead_general_info.lead_memorialized_bool,lead_general_info.lead_multi_locale_phonetic_first_name,lead_general_info.lead_volunteer_causes,lead_general_info.lead_temp_status_emoji,lead_general_info.lead_maiden_name,lead_general_info.lead_geo_location_backfilled_bool,lead_general_info.lead_phonetic_last_name,lead_general_info.lead_premium_bool,lead_general_info.lead_influencer_bool,lead_general_info.lead_headline,lead_general_info.lead_full_name_pronunciation_audio,lead_general_info.lead_summary,lead_general_info.lead_location_name,lead_general_info.lead_address,lead_general_info.lead_headline_location_without_country,lead_general_info.lead_headline_location_with_country,lead_general_info.lead_full_name_pronunciation_audio.urn,lead_general_info.lead_full_name_pronunciation_audio.duration,lead_general_info.lead_full_name_pronunciation_audio.url,lead_general_info.lead_full_name_pronunciation_audio.$type,lead_general_info.lead_birth_date.month,lead_general_info.lead_birth_date.day,lead_general_info.lead_birth_date.$type,lead_general_info.lead_birth_date.year,lead_general_info.lead_multi_locale_phonetic_first_name.en_US,lead_general_info.lead_multi_locale_phonetic_first_name.ja_JP,lead_general_info.lead_state_what.stateCode,lead_general_info.lead_state_what.stateName,lead_general_info.lead_state_what.$type,top_quantil,is_PHD,is_Master,is_MBA,experience_title,experience_start_date,experience_start_date_year,experience_end_date,experience_headcount_range,experience_company_name,experience_company_link,experience_company_id_link,experience_industry,experience_location,experience_description,status,executive_degree,best_degree,person_uuid,org_uuid,job_uuid,started_on,ended_on,is_current,title,job_type,company_name,company_uuid,investor_name,investor_uuid
0,001e860c-6579-9835-b294-af1a65842ca9,,Evan,Rifkin,Evan Rifkin,,41865,,evanrifkin,NPRfcMTmSZ+PC+Of4bNO/w==,,False,,,,,False,,False,False,"Co-Founder, CEO at Aircam",,,,,"Santa Monica, California","Santa Monica, California, United States",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,"Co-Founder, CEO|Project Team Lead|Co-Founder|Co-Founder|Co-Founder|Founding Investor",Jan 2018|Feb 2014|Apr 2009|Dec 2006|May 2005|2005,2018|2014|2009|2006|2005|2005,|Oct 2014|Feb 2014|Dec 2008|Dec 2006|2006,11-50|10001|11-50||51-200|501-1000,"Aircam|Apple|Burstly|Flux, a Social Project|TagWorld|Adconion Media Group",https://www.linkedin.com/company/aircam-inc/|https://www.linkedin.com/company/apple/|https://www.linkedin.com/company/burstly/||https://www.linkedin.com/company/tagworld/|https://www.linkedin.com/company/adconion-media-group/,https://www.linkedin.com/company/19015454|https://www.linkedin.com/company/162479|https://www.linkedin.com/company/1824649|https://www.linkedin.com/company/|https://www.linkedin.com/company/60839|https://www.linkedin.com/company/30596,4|24|80||6|113,|||||,"|||Social Project was sold to MTVN/Viacom in September 2008.\nSocial Project created the Flux Platform, enabling publishers to add social media tools and distribute content across different partner sites of Flux.|TagWorld became Social Project/Flux after an investment from MTVN/Viacom in December 2006.|",entrepreneur,NaN,B,001e860c-6579-9835-b294-af1a65842ca9,997b1e1e-226e-16e1-c977-a0ac3ed940b1,963f8dc0-d8eb-2ab3-e428-ce1c37bd5999,2005-01-01,2006-01-01,f,Founding Investor,board_member,Adconion Media Group,997b1e1e-226e-16e1-c977-a0ac3ed940b1,NaN,NaN
1,0023bf7f-5417-04f4-85cf-5bb078576a57,,Jaynie Miller,Studenmund,Jaynie Miller Studenmund,,83202432,,jaynie-miller-studenmund-680a4523,NN4yJ60NSGe7yg30QoP6gQ==,,False,,,,

In [38]:
new = df.groupby(['status', 'title']).size().reset_index(name = 'count')
new.to_csv('status_position_table.csv', index=False, encoding = 'utf-8-sig')
new

,status,title,count
0,caesar,Acting Chief Executive Officer,1
1,caesar,Administrative Partner and Chief Financial Officer,1
2,caesar,Advisor,50
3,caesar,Advisor / Past Board Director,1
4,caesar,Advisor to Business Team,1
...,...,...,...
2561,investor,founder,1
2562,investor,managing director,1
2563,investor,managing partner,1
2564,investor,principal and a partner,1


Теперь получим все вакансии из LI

In [37]:
df2 = pd.DataFrame()

for dataset in [data, data1, data2]:
    
    g = pd.json_normalize(dataset, record_path='experiences', meta=['uuid'])
    #g = g.merge(ufg[['name', 'degree', 'top_quantil', 'is_PHD', 'is_Master', 'is_MBA']], how = 'left', left_on=['education_school_name', 'education_degree'], right_on=['name', 'degree'])
    #g = g.groupby('uuid').agg(lambda x: '|'.join(list(x.dropna()))).reset_index()
    
    df2 = pd.concat([df2, g], ignore_index=True)

df2.to_csv('df2_experience.csv', index=False, encoding = 'utf-8-sig')
df2 = df2[['uuid', 'experience_company_name', 'experience_title', 'experience_start_date_year','experience_start_date', 'experience_end_date']]
df2

,uuid,experience_company_name,experience_title,experience_start_date_year,experience_start_date,experience_end_date
0,001e860c-6579-9835-b294-af1a65842ca9,Aircam,"Co-Founder, CEO",2018,Jan 2018,
1,001e860c-6579-9835-b294-af1a65842ca9,Apple,Project Team Lead,2014,Feb 2014,Oct 2014
2,001e860c-6579-9835-b294-af1a65842ca9,Burstly,Co-Founder,2009,Apr 2009,Feb 2014
3,001e860c-6579-9835-b294-af1a65842ca9,"Flux, a Social Project",Co-Founder,2006,Dec 2006,Dec 2008
4,001e860c-6579-9835-b294-af1a65842ca9,TagWorld,Co-Founder,2005,May 2005,Dec 2006
...,...,...,...,...,...,...
75740,767846ae-f695-72b5-ba65-f9e50cebf800,Workday,Founder,,,
75741,5ccfa9c5-3c80-596f-9e4a-f76b1a12047e,Mona Vie,Independent Distributor,2008,2008,
75742,5ccfa9c5-3c80-596f-9e4a-f76b1a12047e,Delaware Express Shuttle,part time driver,2007,2007,
75743,5ccfa9c5-3c80-596f-9e4a-f76b1a12047e,School District of Philadelphia,"English Teacher, Track coach",1970,Sep 1970,2004


In [38]:
qwerty = []
q = set(funding_rounds['company_name'].values) #736
q = pd.Series(list(q))
e = set(df2['experience_company_name'].values) #41165
e = list(e)

for j in e:
    if any(q.str.contains(j, regex=False)):
        qwerty.append(j)

for a in ('', ' ', '-', '.'):
    qwerty.remove(a)
    
len(qwerty)

685

In [39]:
df3 = df2.loc[(df2['experience_company_name'].str.contains('|'.join(q))) | (df2['experience_company_name'].isin(qwerty))]
df3

C:\Users\Barca\anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,uuid,experience_company_name,experience_title,experience_start_date_year,experience_start_date,experience_end_date
5,001e860c-6579-9835-b294-af1a65842ca9,Adconion Media Group,Founding Investor,2005,2005,2006
8,0023bf7f-5417-04f4-85cf-5bb078576a57,LifeLock (LOCK),Board Member,2015,2015,2017
31,0029836e-5dec-d285-c440-00d33464ede2,Cloudmark,Founder & Chief Scientist,2001,Sep 2001,Dec 2008
44,00735871-6e7b-dae2-4789-1d6096c490ef,Box,Advisor,2013,Aug 2013,
84,00a553c4-bc02-d894-4b71-c985fd0c0b8d,OmniGuide Holdings Inc.,Chairman Of The Board,2020,Mar 2020,Jan 2021
...,...,...,...,...,...,...
75643,59fa2045-4fdd-993f-eb5a-3ca8d71c866b,Fabriq,Customer Success & Operations Director,2021,Jan 2021,
75659,4cc78deb-d284-4ce1-e677-4869a82e6931,Awarepoint,President and Chief Executive Officer,2012,Mar 2012,May 2018
75674,33e6cf86-1a50-8279-6db7-5117f815204b,Inspire Medical Systems,Board of Directros/Venture Investor,2009,May 2009,
75719,e8405450-2487-b6d9-7ccf-b4cce8113a67,GoodData,CFO,2013,Jul 2013,Feb 2014


Все образования из LI

In [40]:
df4 = pd.DataFrame()

for dataset in [data, data1, data2]:
    
    z = pd.json_normalize(dataset, record_path='educations', meta=['uuid'])
    #z = z.merge(ufg[['name', 'degree', 'top_quantil', 'is_PHD', 'is_Master', 'is_MBA']], how = 'left', left_on=['education_school_name', 'education_degree'], right_on=['name', 'degree'])
    #z = z.groupby('uuid').agg(lambda x: '|'.join(list(x.dropna()))).reset_index()
    
    df4 = pd.concat([df4, z], ignore_index=True)

df4 = df4.merge(ufg[['name', 'degree', 'degree_class']], how = 'left', left_on = ['education_school_name', 'education_degree'], right_on = ['name', 'degree'])
df4 = df4[['uuid', 'education_school_name', 'education_degree', 'degree_class', 'education_start_date', 'education_end_date']]
df4 = df4.loc[df4['education_end_date'] != '']
df4

,uuid,education_school_name,education_degree,degree_class,education_start_date,education_end_date
3,0026bf40-8f42-87e2-b371-978055165959,"University of California, Berkeley, Haas School of Business",MBA,A,2003,2004
4,0026bf40-8f42-87e2-b371-978055165959,Stanford University,BS,B,1993,1997
5,0029836e-5dec-d285-c440-00d33464ede2,"St Stephens College, Delhi University",None,NaN,1995,1997
6,003bbe2e-18be-3c96-e327-18d2295427e0,Harvard College,BA,B,1992,1996
7,003bbe2e-18be-3c96-e327-18d2295427e0,La Jolla Country Day School,,NaN,1988,1992
...,...,...,...,...,...,...
20351,fd8b2b3b-9e2a-900d-ee58-282975891f21,Concord University,,NaN,1986,1989
20352,e8405450-2487-b6d9-7ccf-b4cce8113a67,University of Wisconsin Oshkosh,Bachelor of Business Administration (BBA),B,1977,1981
20353,e8405450-2487-b6d9-7ccf-b4cce8113a67,Hamilton High School,,NaN,1973,1977
20360,5ccfa9c5-3c80-596f-9e4a-f76b1a12047e,Villanova University,B.S.,B,1966,1970


In [41]:
a = df4.loc[df4['degree_class'] == 'B']
a.loc[a['education_end_date'].str.len() == 4, 'education_end_date'] = a['education_end_date'] + '-06-01'
a.loc[a['education_end_date'].str[:3] == 'Jan', 'education_end_date'] = a['education_end_date'].str[-4:] + '-01-01'
a.loc[a['education_end_date'].str[:3] == 'Feb', 'education_end_date'] = a['education_end_date'].str[-4:] + '-02-01'
a.loc[a['education_end_date'].str[:3] == 'Mar', 'education_end_date'] = a['education_end_date'].str[-4:] + '-03-01'
a.loc[a['education_end_date'].str[:3] == 'Apr', 'education_end_date'] = a['education_end_date'].str[-4:] + '-04-01'
a.loc[a['education_end_date'].str[:3] == 'May', 'education_end_date'] = a['education_end_date'].str[-4:] + '-05-01'
a.loc[a['education_end_date'].str[:3] == 'Jun', 'education_end_date'] = a['education_end_date'].str[-4:] + '-06-01'
a.loc[a['education_end_date'].str[:3] == 'Jul', 'education_end_date'] = a['education_end_date'].str[-4:] + '-07-01'
a.loc[a['education_end_date'].str[:3] == 'Aug', 'education_end_date'] = a['education_end_date'].str[-4:] + '-08-01'
a.loc[a['education_end_date'].str[:3] == 'Sep', 'education_end_date'] = a['education_end_date'].str[-4:] + '-09-01'
a.loc[a['education_end_date'].str[:3] == 'Oct', 'education_end_date'] = a['education_end_date'].str[-4:] + '-10-01'
a.loc[a['education_end_date'].str[:3] == 'Nov', 'education_end_date'] = a['education_end_date'].str[-4:] + '-11-01'
a.loc[a['education_end_date'].str[:3] == 'Dec', 'education_end_date'] = a['education_end_date'].str[-4:] + '-12-01'

a['education_end_date'] = pd.to_datetime(a['education_end_date'])
a = a.loc[a.groupby(['uuid'])['education_end_date'].idxmax()]
a

,uuid,education_school_name,education_degree,degree_class,education_start_date,education_end_date
18210,000344fb-6e07-d88b-1608-593f005d38f3,Harvard University,AB,B,1999,2003-06-01
19548,0004aea3-ee6f-5531-558d-c8b50c3afb21,Binghamton University,Bachelor's Degree,B,1985,1989-06-01
7462,001f81b5-a191-03e5-e0a7-44731a87047f,UC San Diego,BA,B,1981,1983-06-01
4,0026bf40-8f42-87e2-b371-978055165959,Stanford University,BS,B,1993,1997-06-01
16620,002f79b2-baf6-d512-ef43-4f3dfc020510,Universidad Metropolitana (VE),Bachelor's degree,B,2006,2014-06-01
...,...,...,...,...,...,...
15781,ffdae148-10a4-226c-8e67-1be6f9a5a496,Harvard University,A.B,B,2007,2011-06-01
7456,ffdfc425-c8da-8a83-32d5-e64d18166bd1,Tel Aviv University,Bsc.,B,1984,1987-06-01
7457,fff3412c-3646-5d39-1ac7-b5aa6a5e6008,"University of California, Berkeley",Bachelor's Degree,B,1980,1983-06-01
15783,fffc4b17-9391-837e-d6f4-7fac3162db06,Bates College,Bachelor of Arts (BA),B,1992,1996-06-01


In [42]:
b = df4.loc[~df4['uuid'].isin(set(a['uuid'].values))]
b.loc[b['education_end_date'].str.len() == 4, 'education_end_date'] = b['education_end_date'] + '-06-01'
b.loc[b['education_end_date'].str[:3] == 'Jan', 'education_end_date'] = b['education_end_date'].str[-4:] + '-01-01'
b.loc[b['education_end_date'].str[:3] == 'Feb', 'education_end_date'] = b['education_end_date'].str[-4:] + '-02-01'
b.loc[b['education_end_date'].str[:3] == 'Mar', 'education_end_date'] = b['education_end_date'].str[-4:] + '-03-01'
b.loc[b['education_end_date'].str[:3] == 'Apr', 'education_end_date'] = b['education_end_date'].str[-4:] + '-04-01'
b.loc[b['education_end_date'].str[:3] == 'May', 'education_end_date'] = b['education_end_date'].str[-4:] + '-05-01'
b.loc[b['education_end_date'].str[:3] == 'Jun', 'education_end_date'] = b['education_end_date'].str[-4:] + '-06-01'
b.loc[b['education_end_date'].str[:3] == 'Jul', 'education_end_date'] = b['education_end_date'].str[-4:] + '-07-01'
b.loc[b['education_end_date'].str[:3] == 'Aug', 'education_end_date'] = b['education_end_date'].str[-4:] + '-08-01'
b.loc[b['education_end_date'].str[:3] == 'Sep', 'education_end_date'] = b['education_end_date'].str[-4:] + '-09-01'
b.loc[b['education_end_date'].str[:3] == 'Oct', 'education_end_date'] = b['education_end_date'].str[-4:] + '-10-01'
b.loc[b['education_end_date'].str[:3] == 'Nov', 'education_end_date'] = b['education_end_date'].str[-4:] + '-11-01'
b.loc[b['education_end_date'].str[:3] == 'Dec', 'education_end_date'] = b['education_end_date'].str[-4:] + '-12-01'

b['education_end_date'] = pd.to_datetime(b['education_end_date'])
b = b.loc[b.groupby(['uuid'])['education_end_date'].idxmax()]
b

,uuid,education_school_name,education_degree,degree_class,education_start_date,education_end_date
5,0029836e-5dec-d285-c440-00d33464ede2,"St Stephens College, Delhi University",None,NaN,1995,1997-06-01
7492,009afe69-2ca9-055e-a0d1-312ea11df919,University of Oxford,,NaN,2018,2018-06-01
23,00ed15b2-7b25-df6d-e83b-57571e765945,"NYU, Skirball Institute, NYC",,NaN,1999,2001-06-01
7517,0139cdfe-3991-c114-cbeb-dbd7507ac741,WHU – Otto Beisheim School of Management,,NaN,2005,2008-06-01
19126,01632a94-81a1-9ea5-94ae-d1d94450063a,University of Illinois Chicago,Pharm D,D,1994,1998-06-01
...,...,...,...,...,...,...
15740,fefc98d1-7c85-cc6e-5687-18ba000d9e47,PSG College of Technology,MSc,M,2000,2005-06-01
15743,fefe48e6-3c2a-9d21-c51e-0c0f98f716a1,ESSEC Business School,,NaN,1989,1992-06-01
15757,ff436b49-11d8-6e67-3a4a-e2c28815d262,London Business School,Executive MBA,E,2008,2010-06-01
7442,ff4be7ff-a242-598e-7a62-8a05e44df9e5,MIT Sloan School of Management,Sloan Fellow,NaN,2004,2006-06-01


In [43]:
df4 = pd.concat([a, b], ignore_index=True)

In [44]:
df2 = df2.loc[df2['experience_start_date'] != '']
df2.loc[df2['experience_start_date'].str.len() == 4, 'experience_start_date'] = df2['experience_start_date'] + '-01-01'
df2.loc[df2['experience_start_date'].str[:3] == 'Jan', 'experience_start_date'] = df2['experience_start_date'].str[-4:] + '-01-01'
df2.loc[df2['experience_start_date'].str[:3] == 'Feb', 'experience_start_date'] = df2['experience_start_date'].str[-4:] + '-02-01'
df2.loc[df2['experience_start_date'].str[:3] == 'Mar', 'experience_start_date'] = df2['experience_start_date'].str[-4:] + '-03-01'
df2.loc[df2['experience_start_date'].str[:3] == 'Apr', 'experience_start_date'] = df2['experience_start_date'].str[-4:] + '-04-01'
df2.loc[df2['experience_start_date'].str[:3] == 'May', 'experience_start_date'] = df2['experience_start_date'].str[-4:] + '-05-01'
df2.loc[df2['experience_start_date'].str[:3] == 'Jun', 'experience_start_date'] = df2['experience_start_date'].str[-4:] + '-06-01'
df2.loc[df2['experience_start_date'].str[:3] == 'Jul', 'experience_start_date'] = df2['experience_start_date'].str[-4:] + '-07-01'
df2.loc[df2['experience_start_date'].str[:3] == 'Aug', 'experience_start_date'] = df2['experience_start_date'].str[-4:] + '-08-01'
df2.loc[df2['experience_start_date'].str[:3] == 'Sep', 'experience_start_date'] = df2['experience_start_date'].str[-4:] + '-09-01'
df2.loc[df2['experience_start_date'].str[:3] == 'Oct', 'experience_start_date'] = df2['experience_start_date'].str[-4:] + '-10-01'
df2.loc[df2['experience_start_date'].str[:3] == 'Nov', 'experience_start_date'] = df2['experience_start_date'].str[-4:] + '-11-01'
df2.loc[df2['experience_start_date'].str[:3] == 'Dec', 'experience_start_date'] = df2['experience_start_date'].str[-4:] + '-12-01'

df2['experience_start_date'] = pd.to_datetime(df2['experience_start_date'])
df2 = df2.loc[df2.groupby(['uuid'])['experience_start_date'].idxmin()]
df2

,uuid,experience_company_name,experience_title,experience_start_date_year,experience_start_date,experience_end_date
68602,000344fb-6e07-d88b-1608-593f005d38f3,McKinsey & Company,Business Analyst,2004,2004-09-01,May 2006
72977,0004aea3-ee6f-5531-558d-c8b50c3afb21,Morgan Stanley,Analyst,1989,1989-07-01,Sep 1991
30095,00122598-4cf3-0a67-a91e-b1df72d8ee46,Trinity Ventures,General Partner,1998,1998-01-01,2008
30102,0014409b-f138-ecd1-31ad-6096989254c3,Goldman Sachs,Investment Banking,2007,2007-01-01,2007
5,001e860c-6579-9835-b294-af1a65842ca9,Adconion Media Group,Founding Investor,2005,2005-01-01,2006
...,...,...,...,...,...,...
66743,ffdc517c-ca48-4674-9b8b-4635a36b72af,Bundeswehr (German Federal Armed Forces),Infantry Soldier,1992,1992-03-01,Mar 1993
30080,ffdfc425-c8da-8a83-32d5-e64d18166bd1,Skinos Family Office,Investor,1999,1999-01-01,
30085,fff3412c-3646-5d39-1ac7-b5aa6a5e6008,IBM,Manager,1983,1983-01-01,1991
59826,fffc4b17-9391-837e-d6f4-7fac3162db06,The Home for Little Wanderers,Human Resources,1996,1996-01-01,2002


Следующий блок превращает датафрейм выше в разреженную таблицу one-hot векторов. То есть каждая строка так же отвечает конкретному человеку, а в виде столбцов для каждого квинтиля места учебы и ученой степени проставлен флаг наличия этого категориального признака у человека.  
Так как от людей мы хотим перейти к инвестиционным раундам, нам надо будет преобразовать вектора для людей в вектора для раундов. Проблема в том, что для каждой компании известно разное количество человек, в ней работавших, поэтому придется нормализовать вектора с учетом этого. Схема следующая: для каждой компании производится группировка - вектора складываются между собой, а затем производится деление на количество векторов == людей. Кроме этого, на момент инвестиционного события не все числившиеся в компании люди актуальны, так что для каждой строки == раунда выбираем тех людей из компании, в стаже которых в ней имеется период в течение года до раунда.

In [45]:
one_hot_df = df.loc[~df['company_uuid'].isin(nan)]
one_hot_df = one_hot_df[['uuid', 'lead_general_info.lead_full_name', 'company_uuid', 'company_name', 'job_type', 'title', 'started_on', 'ended_on', 'top_quantil', 'is_PHD', 'is_Master', 'is_MBA']]
one_hot_df = one_hot_df.drop_duplicates().reset_index(drop=True)
one_hot_df['total_count'] = one_hot_df['company_uuid'].map(one_hot_df['company_uuid'].value_counts())

#переход от людей (6427 с учетом нескольких должностей в одной компании, иначе 5972) к раундам (3733) с размножением
one_hot_df = funding_rounds.merge(one_hot_df, how = 'left', on = ['company_uuid', 'company_name'])
#one_hot_df = one_hot_df.merge(df3, how = 'left', on = 'uuid') #информация с LI не слишком релевантна и менее полна

#даты
one_hot_df['announced_on'] = pd.to_datetime(one_hot_df['announced_on'])
one_hot_df['started_on'] = pd.to_datetime(one_hot_df['started_on'])
one_hot_df['ended_on'] = pd.to_datetime(one_hot_df['ended_on'])
#сейчас 36197 раундов
a = one_hot_df.loc[((one_hot_df['started_on'] <= one_hot_df['announced_on']) | one_hot_df['started_on'].isin(nan)) & ((one_hot_df['ended_on'] >= (one_hot_df['announced_on'] - pd.DateOffset(years=1))) | one_hot_df['ended_on'].isin(nan))]
a = a.groupby(['funding_round_uuid', 'company_uuid']).size().reset_index(name='count')
b = one_hot_df.groupby(['funding_round_uuid', 'company_uuid']).size().reset_index()
b = b.merge(a, how = 'left', on = ['funding_round_uuid', 'company_uuid'])
b = b.loc[b['count'].isin(nan)]
b = one_hot_df.merge(b[['funding_round_uuid', 'company_uuid']], how = 'right', on = ['funding_round_uuid', 'company_uuid'])
b = b.drop_duplicates(subset=['funding_round_uuid', 'company_uuid']).reset_index(drop=True)
for i in ['top_quantil', 'is_PHD', 'is_Master', 'is_MBA', 'total_count']:
    b[i] = 0
#нашли 8 раундов, которые удалятся полностью из-за отсутствия людей, оставим их с нулями
one_hot_df = one_hot_df.loc[((one_hot_df['started_on'] <= one_hot_df['announced_on']) | one_hot_df['started_on'].isin(nan)) & ((one_hot_df['ended_on'] >= (one_hot_df['announced_on'] - pd.DateOffset(years=1))) | one_hot_df['ended_on'].isin(nan))]
one_hot_df = pd.concat([one_hot_df, b], ignore_index=True)
#выбросили 7061, неподходящих по датам (минус 8 раундов), и вернули их обратно с нулями
one_hot_df = one_hot_df.drop_duplicates(subset=['funding_round_uuid', 'company_uuid', 'uuid']).reset_index(drop=True)
#убрали повторения людей в одном раунде

#стаж
one_hot_df = one_hot_df.merge(df4[['uuid', 'degree_class', 'education_end_date']], how = 'left', on = 'uuid')
one_hot_df = one_hot_df.merge(df2[['uuid', 'experience_start_date']], how = 'left', on = 'uuid')
one_hot_df['min_start_date'] = one_hot_df[['education_end_date', 'experience_start_date']].min(axis=1)
one_hot_df['announced_on'] = pd.to_datetime(one_hot_df['announced_on'])
one_hot_df['experience_in_months'] = ((one_hot_df['announced_on'] - one_hot_df['min_start_date'])/np.timedelta64(1, 'M'))
one_hot_df.loc[(one_hot_df['experience_in_months'].isin(nan)) | (one_hot_df['experience_in_months'] < 0), 'experience_in_months'] = 0
one_hot_df['count_per_round'] = one_hot_df.groupby(['funding_round_uuid', 'company_uuid'])['company_uuid'].transform('count')

#перевод в one-hot (отключено) и схлопывание по раундам

columns_before = list(one_hot_df.columns)
columns_before.remove('top_quantil')
columns_before.remove('is_PHD')
columns_before.remove('is_Master')
columns_before.remove('is_MBA')
#one_hot_df = pd.concat([one_hot_df.drop('top_quantil', 1), one_hot_df['top_quantil'].str.get_dummies(sep="|").add_prefix('EDU_')], 1)
#columns_after = list(one_hot_df.columns)
#listColumns = list(set(columns_after) - set(columns_before))
listColumns = ['top_quantil', 'is_PHD', 'is_Master', 'is_MBA']
d1 = {k:'sum' for k in listColumns}
d2 = {'experience_in_months': ['min', 'max', 'mean', 'std']}
d1.update(d2)
one_hot_df = one_hot_df.replace(np.nan, '---')
for column in ['uuid', 'lead_general_info.lead_full_name', 'job_type', 'title', 'started_on', 'ended_on', 'degree_class', 'education_end_date', 'experience_start_date', 'min_start_date', 'experience_in_months']:
    columns_before.remove(column)

#агрегирование данных по людям в один раунд
one_hot_df = one_hot_df.groupby(columns_before).agg(d1).sort_values(by = ['company_uuid', 'announced_on']).reset_index()
one_hot_df[listColumns] = one_hot_df[listColumns].apply(pd.to_numeric, errors='coerce')
#one_hot_df.loc[:, listColumns] = one_hot_df.loc[:, listColumns].div(one_hot_df['count_per_round'], axis=0)

one_hot_df.to_csv('people_one_hot.csv', index=False, encoding = 'utf-8-sig')
one_hot_df

funding_round_uuid company_name  \
                                                          
0     778db45f-e9d1-3f73-86f4-ae4cc79c48ff        Lot18   
1     fc2c2560-8bb0-90d1-b048-ee9cfc3f301d        Lot18   
2     d863a1b9-dda0-284e-3258-fa43a20d305c        Lot18   
3     4c9cb685-ec9a-92ed-64c0-062040d9573e        Lot18   
4     1006ef75-d2f2-bac4-5737-dae2075cf616        Lot18   
...                                    ...          ...   
3728  80f5449a-2b38-f3eb-dc4f-d5b581e636ec    ZS Pharma   
3729  e79c29e1-b5a7-db70-014d-36c2ba16338b    ZS Pharma   
3730  2067db69-a274-7914-0207-5fb1af6cc7ca    ZS Pharma   
3731  602347ee-2d81-7ad3-7f85-d68ea1fb8967    ZS Pharma   
3732                                     -    ZS Pharma   

                              company_uuid investment_type announced_on  \
                                                                          
0     003d89d4-47dc-e35a-ddf6-eda9940f3ba7            seed   2010-01-01   
1     003d89d4-47dc-e35a-ddf6-eda9940f3ba7        series_a   2010-11-10   
2     003d89d4-47dc-e35a-ddf6-eda9940f3ba7        series_b   2011-05-02   
3     003d89d4-47dc-e35a-ddf6-eda9940f3ba7  series_unknown   2011-08-24   
4     003d89d4-47dc-e35a-ddf6-eda9940f3ba7        series_c   2011-11-04   
...                                    ...             ...          ...   
3728  ffd3e2ec-f764-6135-4b41-9a861965223d  series_unknown   2010-12-14   
3729  ffd3e2ec-f764-6135-4b41-9a861965223d  series_unknown   2010-12-20   
3730  ffd3e2ec-f764-6135-4b41-9a861965223d        series_c   2012-10-23   
3731  ffd3e2ec-f764-6135-4b41-9a861965223d        series_d   2014-03-05   
3732  ffd3e2ec-f764-6135-4b41-9a861965223d             ipo   2014-06-19   

     raised_amount_usd post_money_valuation_usd   exit_date exit_type  \
                                                                        
0               500000                      ---         ---       ---   
1                3e+06                      ---         ---       ---   
2                1e+07                      ---         ---       ---   
3                1e+06                      ---         ---       ---   
4                3e+07                      ---         ---       ---   
...                ...                      ...         ...       ...   
3728       1.19597e+06                      ---  2014-06-19       ipo   
3729            792000                      ---  2014-06-19       ipo   
3730           4.6e+07                      ---  2014-06-19       ipo   
3731           5.5e+07                      ---  2014-06-19       ipo   
3732          1.12e+08                    3e+08  2014-06-19       ipo   

     price_usd money_raised_usd time_from_2009 time_from_start  \
                                                                 
0          ---              ---          365.0             0.0   
1          ---              ---          678.0           313.0   
2          ---              ---          851.0           486.0   
3          ---              ---          965.0           600.0   
4          ---              ---         1037.0           672.0   
...        ...              ...            ...             ...   
3728     3e+08         1.12e+08          712.0             0.0   
3729     3e+08         1.12e+08          718.0             6.0   
3730     3e+08         1.12e+08         1391.0           679.0   
3731     3e+08         1.12e+08         1889.0          1177.0   
3732     3e+08         1.12e+08         1995.0          1283.0   

     time_from_previous top_investor other_investor top_investor_count  \
                                                                         
0                   ---          0.0            0.0                0.0   
1                   313          1.0            0.0                3.0   
2                   173          1.0            0.0                3.0   
3                   114          0.0            1.0                3.0   
4                    72    